# Examples 

As our main examples we will consider Hilbert modular groups of the following three number Fields:
- $ K_1 = \mathbb{Q}(\sqrt{5})$  (one cusp, discriminant 5, generally the simplest example)
- $ K_2 = \mathbb{Q}(\sqrt{10})$ (two cusps, discriminant 40)
- $ K_3 = \mathbb{Q}(\alpha)$, where $\alpha$ has minimal polynomial $\alpha^3-\alpha^2-2x+1$ (one cusp, discriminant 49)

In this notebook we will look at the first example. 

Note that SageMath chooses the fundamental unit $\epsilon=-(1+\sqrt{5})/2$ but this choice does not effect any of the results.

## Introductory example of basic usage
Let's first demonstrate the reduction of a point $$z=(0.1+i,0.2+0.5i) \in \mathbb{H}^2$$ with respect to the fundamental domain of $K_1$.

In [ ]:
from plot import plot_polygon
from hilbert_modgroup.all import HilbertModularGroup, HilbertPullback, UpperHalfPlaneProductElement
H1 = HilbertModularGroup(5)
K1 = H1.base_ring().number_field()
P1 = HilbertPullback(H1)

In [ ]:
z = UpperHalfPlaneProductElement([0.1+I,0.2+0.5*I])
P1.reduce(z)

Then check that if we apply a random element of $H_1$ we get the same (up to numerical error) reduced point

In [ ]:
w = H1.random_element(-5,5).acton(z)
P1.reduce(z) - P1.reduce(w)

## Details of the algorithm
We will now go through the inner workings of the algorithm and the construction of the relevant bounds in more detail. Note that the functions below will mainly be of interest to researchers who wants to verify or extend these algorithms or results.

In [ ]:
K1.discriminant()

$$
B_{\Lambda} = (\begin{array}{cc}
\log(\frac{1}{2}(1+\sqrt{5})) & \log(\frac{1}{2}(\sqrt{5}-1))\end{array})^t
$$

In [ ]:
P1.basis_matrix_logarithmic_unit_lattice()

In [ ]:
P1.basis_matrix_logarithmic_unit_lattice().norm(Infinity)

$$
B_{\mathcal{O}_K}
=\left(\begin{array}{cc}
1 & -\frac{1}{2}\left(1+\sqrt{5}\right)\\
1 & -\frac{1}{2}\left(1-\sqrt{5}\right)
\end{array}\right),\ 
B_{\mathcal{O}_K}^{-1}=\frac{1}{\sqrt{5}}\left(\begin{array}{cc}
-\frac{1}{2}(1-\sqrt{5}) & \frac{1}{2}(1+\sqrt{5})\\
-1 & 1
\end{array}\right)
$$

In [ ]:
P1.D() # D_0 and D_1

In [ ]:
P1.basis_matrix_ideal().inverse()

In [ ]:
P1.basis_matrix_ideal().norm(Infinity)

In [ ]:
P1.basis_matrix_ideal().inverse().norm(Infinity)

Consider the point $\mathbf{z}=i\mathbf{1}$.

In [ ]:
z=UpperHalfPlaneProductElement([CC(0,1),CC(0,1)])

Make a plot of the box given by the bounds of the embeddings together with the curves bounding the norm.

The norm bound implies that  $\sigma_1 \sigma_2 \le 1.0$

In [ ]:
P1._bound_for_sigma_norm(z)

In [ ]:
P1.get_heuristic_closest_cusp(z)

In [ ]:
# Add arguments to include the norm bounds
norm_args = {'norm_bound':P1._bound_for_sigma_norm(z)}

In [ ]:
p=P1._candidate_integers_sigma(z,domain='preimage',return_polyhedron=True)

In [ ]:
plot_polygon(p,K1.ideal(1).integral_basis(),action='show',**norm_args)

We observe here that of course the points $1$ and $-1$ lie on the norm bound curves. 

In [ ]:
plot_polygon(p,K1.ideal(1).integral_basis(),action='save',filename='K1.z1.domain1.pgf',**norm_args)

In [ ]:
p1=P1._candidate_integers_sigma(z,domain='polytope',return_polyhedron=True)

In [ ]:
# Add arguments to include the norm bounds
norm_args = {'norm_bound':P1._bound_for_sigma_norm(z),'curve_map':P1.basis_matrix_ideal().inverse()}

In [ ]:
plot_polygon(p1,[1,1],action='show',**norm_args)

In [ ]:
plot_polygon(p1,[1,1],action='save',filename='K1.z1.domain2.pgf',**norm_args)

Now let's see what candidates we have for sigma:

In [ ]:
l=P1._candidate_integers_sigma(z); l

And the candidates for closest cusps are (1:0), (0:1), (1:1) and (1:-1):

In [ ]:
for c in P1._candidate_closest_cusps(z):
    print(f"({str(c[0]):<3} : {str(c[1]):>3})",P1.distance_to_cusp(c,z))

Let's go through the same steps but for $\mathbf{z}=\frac{i}{2} \mathbf{1}$:

In [ ]:
z=UpperHalfPlaneProductElement([CC(0,0.5),CC(0,0.5)])

If we try to find a preliminary bound we find easily that the distance to the cusp at 0 is 0.5 and the distance to infinity is 2.0 

In [ ]:
print("distance to 0/1:",P1.distance_to_cusp(NFCusp(K1,0,1),z))
print("distance to 1/0:",P1.distance_to_cusp(NFCusp(K1,1,0),z))


We can use a $d=0.5$ and in fact obtain the same bounds as above: 

In [ ]:
P1._bound_for_sigma_embeddings(z)

Which obviously yields the same candidates for $\sigma$

In [ ]:
P1._candidate_integers_sigma(z)

But the candidates for $\rho$ differs and we actually only have $\rho=0$

In [ ]:
for s in P1._candidate_integers_sigma(z):
    print(P1._bound_for_rho_embeddings(z,s))
    print(f"{s}: {P1._candidate_integers_rho(z,s)}")

In [ ]:
for c in P1._candidate_closest_cusps(z):
    print(f"({str(c[0]):<3} : {str(c[1]):>3})",P1.distance_to_cusp(c,z))

And the closest cusp is indeed $(0:1)$

Now, just for illustration, let's see what happens if we don't try to find any preliminary bound but just go with infinity as initial cusp. We then get twice the original bounds for $\sigma$:

In [ ]:
P1._bound_for_sigma_embeddings(z,use_initial_bd_d=False)

In [ ]:
# The norm bound with d= dist to oo
P1._bound_for_sigma_norm(z)

In [ ]:
basis=K1.ideal(1).integral_basis(); basis

In [ ]:
P1._bound_for_sigma_embeddings(z,use_initial_bd_d=False)

In [ ]:
p1=P1._candidate_integers_sigma(z,domain='preimage',return_polyhedron=True,use_initial_bd_d=False)
norm_args = {'norm_bound':P1._bound_for_sigma_norm(z)}
plot_polygon(p1,basis,action='show',ticks=[2,2],xmin=-3,xmax=3,ymin=-3,ymax=3,**norm_args)

In [ ]:
plot_polygon(p1,K1.ideal(1).integral_basis(),action='save',filename='K1.z2.domain1.pgf',xmin=-3,xmax=3,ymin=-3,ymax=3,
             ticks=[2,2],**norm_args)

In [ ]:
p=P1._candidate_integers_sigma(z,domain='polytope',return_polyhedron=True,use_initial_bd_d=False)
# Add arguments to include the norm bounds
norm_args = {'norm_bound':P1._bound_for_sigma_norm(z,4),'curve_map':P1.basis_matrix_ideal().inverse()}
plot_polygon(p,[1,1],action='show',ticks=[2,2],xmax=4,xmin=-4,ymin=-4,ymax=4,**norm_args)

In [ ]:
plot_polygon(p,[1,1],action='save',filename='K1.z2.domain2.pgf',ticks=[2,2],xmax=4.5,**norm_args)

We see immediately that in this case we have 9 candidates for $\sigma$

In [ ]:
len(P1._candidate_integers_sigma(z,use_initial_bd_d=False))

And we actually obtain 9 distinct candidates for closest cusp:

In [ ]:
P1._candidate_closest_cusps(z,use_initial_bd_d=False,as_cusps=True)